In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('/home/hv2237/bart')

OSError: /home/hv2237/bart_rl/step_1500 does not appear to have a file named config.json. Checkout 'https://huggingface.co//home/hv2237/bart_rl/step_1500/None' for available files.

In [6]:
path = '/home/hv2237/GYAFC_Corpus/Entertainment_Music'
data = {}
        
for split in ['tune', 'test']:
    data[split] = []
    refs = [open(f'{path}/{split}/formal.ref{i}').readlines() for i in range(4)]
    inp = open(f'{path}/{split}/informal').readlines()
    for f in range(len(inp)):
        temp = {}
        temp['informal'] = inp[f][:-1]
        for i in range(4):
            temp[f'formal.ref{i}'] = refs[i][f][:-1]
        data[split].append(temp)

In [7]:
data['tune'][0]

{'informal': 'Any movie that has vampires I like!',
 'formal.ref0': 'I like any movie that has vampires in it.',
 'formal.ref1': 'I enjoy any film that has vampires in it!',
 'formal.ref2': 'I enjoy any film that has vampires.',
 'formal.ref3': 'I like any movie with vampires!'}

In [9]:
import json
for split in ['tune', 'test']:
    json.dump(data[split], open(f'eval_{split}.json','w'))


In [11]:
from datasets import Dataset, load_dataset
eval_dataset = load_dataset('json',data_files={
    'valid':'eval_tune.json',
    'test':'eval_test.json'
})

Using custom data configuration default-af09d24aeef2f376


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/hv2237/.cache/huggingface/datasets/json/default-af09d24aeef2f376/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def preprocess_function_eval(examples):
    # print(examples)
    inputs = [ex for ex in examples["informal"]]
    targets = [ex for ex in examples["formal.ref0"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    for i in range(4):
        model_inputs[f'formal{i}'] = examples[f'formal.ref{i}']
    return model_inputs

In [14]:
max_length = 30
tokenized_datasets_eval = eval_dataset.map(
    preprocess_function_eval,
    batched=True,
    remove_columns=eval_dataset["valid"].column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
def collate(batch):
    keys = list(batch[0].keys())
    cols = {key:[] for key in keys}
    temp = list(map(lambda x: {'input_ids':x['input_ids'], 'labels':x['labels'], 'attention_mask':x['attention_mask']}, batch))
    for i in batch:
        for key in i:
            cols[key].append(i[key])
    out = data_collator(temp)
    for i in range(4):
        out[f'formal{i}'] = cols[f'formal{i}']
    return out

In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
from torch.utils.data import DataLoader

tokenized_datasets_eval.set_format("torch")

eval_dataloader = DataLoader(
    tokenized_datasets_eval["valid"],collate_fn=lambda x: collate(x) ,  batch_size=32
)

In [19]:
test_dataloader = DataLoader(
    tokenized_datasets_eval["test"],collate_fn=lambda x: collate(x) ,  batch_size=32
)

In [27]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [34]:
from accelerate import Accelerator

accelerator = Accelerator()
model, eval_dataloader, test_dataloader = accelerator.prepare(
    model, eval_dataloader, test_dataloader
)

In [22]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [70]:
from tqdm.auto import tqdm
import numpy as np
import evaluate
from nltk.tokenize import word_tokenize

metric_bleu = evaluate.load('bleu')
predictions, references = [], []
model.eval()
for batch in tqdm(eval_dataloader):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=30,
            num_beams=10
        )
    labels = batch["labels"]
    
    # Necessary to pad predictions and labels for being gathered
    generated_tokens = accelerator.pad_across_processes(
        generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
    )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
    predictions += [i for i in decoded_preds]
    # for ref in batch
    # for i in range(1,4):
    transfrom = lambda x: [i for i in x]
    references += list(map(list, zip(*map(transform, [batch[f'formal{i}'] for i in range(4)]))))
    # metric_bleu.add_batch(predictions=decoded_preds, references=decoded_labels)
    # metric_bleurt.add_batch(predictions=decoded_preds, references=list(map(lambda x: x[0],decoded_labels)))
    # for i in range(1,4):
    #     metric_bleu.add_batch(predictions=decoded_preds, references=[[i] for i in batch[f'formal{i}']])
        # metric_bleurt.add_batch(predictions=decoded_preds, references=[i for i in batch[f'formal{i}']])
    
results_bleu = metric_bleu.compute(predictions=predictions, references=references, tokenizer=word_tokenize)
# results_bleurt = metric_bleurt.compute()
# bleurt = np.mean(results_bleurt['scores'])
print(f"BLEU score: {results_bleu['bleu']:.2f}")

metric_bleu = evaluate.load('bleu')
predictions, references = [], []

for batch in tqdm(test_dataloader):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=30,
        )
    labels = batch["labels"]

    # Necessary to pad predictions and labels for being gathered
    generated_tokens = accelerator.pad_across_processes(
        generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
    )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
    predictions += [i for i in decoded_preds]
    # for ref in batch
    # for i in range(1,4):
    transfrom = lambda x: [i for i in x]
    references += list(map(list, zip(*map(transform, [batch[f'formal{i}'] for i in range(4)]))))

    # metric_bleu.add_batch(predictions=decoded_preds, references=decoded_labels)
    # metric_bleurt.add_batch(predictions=decoded_preds, references=list(map(lambda x: x[0],decoded_labels)))
    # for i in range(1,4):
        # metric_bleu.add_batch(predictions=decoded_preds, references=[[i] for i in batch[f'formal{i}']])
        # metric_bleurt.add_batch(predictions=decoded_preds, references=[i for i in batch[f'formal{i}']])

# results_bleu = metric_bleu.compute()
results_bleu = metric_bleu.compute(predictions=predictions, references=references, tokenizer=word_tokenize)

# results_bleurt = metric_bleurt.compute()
# bleurt = np.mean(results_bleurt['scores'])
print(f"BLEU score: {results_bleu['bleu']:.2f}")

  0%|          | 0/90 [00:00<?, ?it/s]

BLEU score: 0.72


  0%|          | 0/45 [00:00<?, ?it/s]

BLEU score: 0.75


In [69]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hv2237/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [65]:
transform = lambda x: [i for i in x]

list(map(list, zip(*map(transform, [batch[f'formal{i}'] for i in range(4)]))))

[['It is a great magazine that has good information without having inappropriate things.',
  "It's a great magazine. It has great information, without anything inappropriate.",
  'It is a great magazine with good information and no inappropriate material.',
  'It is a good magazine. It has good information about inappropriate topics.'],
 ['I do not dislike anyone and neither should you.',
  "I don't hate anybody, neither should you.",
  'I do not hate anyone, and neither should you.',
  'I do not hate anyone and neither should you.'],
 ['"The Real World", "The Newlyweds" and "Laguna Beach".',
  'I enjoy Laguna Beach, Newlyweds and The Real World programs.',
  '"The Real World," "The Newlyweds," and "Laguna Beach."',
  '"The Real World," "Newlyweds," and "Laguna Beach."'],
 ['I think he is too cocky, which annoys me.',
  'He is a very cocky person.',
  'I think that he is too sure of himself and that agitates me.',
  'He is too cocky and that annoys me.'],
 ['The woman looks up and call

In [49]:
list(map(transform, [batch[f'formal{i}'] for i in range(4)]))[0]

[[['We will have an enjoyable time.'],
  ['Yes, a PlayStation Portable, but I would not purchase one for a child under the age of thirteen.'],
  ['If you want to live, talk, and dream about it, then I suggest moving to Japan or China.'],
  ['I very much enjoy it. I will keep this unpleasant secret to myself.'],
  ['I believe that it is the blonde from American Idol.'],
  ['No offence, but if you are able to answer that, then it is the same for the egg.'],
  ['The song was called "Epic"; I can not remember the artist, but it is a great song!'],
  ['Mar, I thought I would let you know that that commercial is for Dairy Queen.']],
 [["We'll have a real good time!"],
  ['Yeah, a PSP but I would not et a PSP for a kid under thirteen.'],
  ['If you want to live and talk and dream about I suggest moving to Japan or China.'],
  ["I love it and I'll keep you my dirty little secret."],
  ['I guess this is the blond from "American Idol."'],
  ['No offense, but if that is your answer, then the same